In [ ]:
from transformers import pipeline

In [ ]:
import re

In [ ]:
from transformers import BertTokenizer, BertModel
import torch

In [ ]:
from nltk.corpus import stopwords

In [ ]:
import nltk

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
def preprocess_text(text):
  stop_words = set(stopwords.words('english'))
  tokens = tokenizer.tokenize(text)
  tokens = [token for token in tokens if token not in stop_words]
  return ' '.join(tokens)

In [ ]:
def encode_text(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).detach().numpy()


In [ ]:
def calculate_similarity(resume_text, job_desc_text):
    resume_encoding = encode_text(preprocess_text(resume_text))
    job_desc_encoding = encode_text(preprocess_text(job_desc_text))
    similarity = cosine_similarity(resume_encoding, job_desc_encoding)
    return similarity[0][0]


In [ ]:
def suggest_improvements(resume_text, job_desc_text):
    resume_tokens = set(tokenizer.tokenize(preprocess_text(resume_text)))
    job_desc_tokens = set(tokenizer.tokenize(preprocess_text(job_desc_text)))
    missing_keywords = job_desc_tokens - resume_tokens
    return list(missing_keywords)

In [ ]:
def analyze_resume(resume_text, job_desc_text):
    score = calculate_similarity(resume_text, job_desc_text)
    improvements = suggest_improvements(resume_text, job_desc_text)
    return score, improvements

In [ ]:
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

In [ ]:
summarized_content = summarizer(
    "PASSIONS:\n"
    "As a newcomer to the industry, I bring a fresh perspective and a genuine passion for learning and growth. "
    "My journey into the world of data and technology began with an insatiable curiosity and a determination to make an impact. "
    "I am characterized by a strong work ethic and a proactive approach to challenges. My willingness to learn and adapt enables me to quickly grasp new concepts and technologies, making me a valuable asset to any team. "
    "While I may be new to the industry, my genuine enthusiasm for innovation and problem-solving drives me, qualities that I am eager to leverage in a professional setting.\n\n"
    "EDUCATION:\n"
    "Bachelor of Technology in Artificial Intelligence\n"
    "Shiv Nadar University"
)

In [ ]:
summarized_content

[{'summary_text': ' As a newcomer to the industry, I bring a fresh perspective and a genuine passion for learning and growth . I am characterized by a strong work ethic and a proactive approach to challenges . My willingness to learn and adapt enables me to quickly grasp new concepts and technologies, making me a valuable asset to any team .'}]

In [ ]:
content = summarized_content[0]['summary_text'].strip()
print(content)

As a newcomer to the industry, I bring a fresh perspective and a genuine passion for learning and growth . I am characterized by a strong work ethic and a proactive approach to challenges . My willingness to learn and adapt enables me to quickly grasp new concepts and technologies, making me a valuable asset to any team .


In [ ]:
pattern = r'\b[a-zA-Z0-9._%+-]+@gmail\.com\b'

# Extract all Gmail addresses
gmail_addresses = re.findall(pattern, text)

# Print the extracted Gmail addresses
print("Extracted Gmail addresses:", gmail_addresses)

In [ ]:
def extract_gmail_addresses(content):
    pattern = r'\b[a-zA-Z0-9._%+-]+@gmail\.com\b'
    gmail_addresses = re.findall(pattern, content)
    return gmail_addresses

In [ ]:
def extract_phone_numbers(content):
    phone_pattern = r'\b\d{10}\b|\b(?:\d{3}[-.\s]?){2}\d{4}\b'
    phone_numbers = re.findall(phone_pattern, content)
    return phone_numbers

In [ ]:
def extract_education(content):
    ner_pipeline = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english")
    entities = ner_pipeline(content)
    education_info = [entity['word'] for entity in entities if entity['entity'] == 'I-ORG' or entity['entity'] == 'B-ORG']
    return education_info

In [ ]:
def extract_contact_info(content):
    contact_info = {}
    contact_info['gmail_addresses'] = extract_gmail_addresses(content)
    contact_info['phone_numbers'] = extract_phone_numbers(content)
    contact_info['education_info'] = extract_education(content)
    return contact_info

In [ ]:
text = """
John Doe
Email: john.doe@gmail.com
Phone: 123-456-7890
Bachelor of Technology in Artificial Intelligence
Shiv Nadar University
"""

# Extract contact information
contact_info = extract_contact_info(text)
print("Extracted Contact Information:", contact_info)

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Extracted Contact Information: {'gmail_addresses': ['john.doe@gmail.com'], 'phone_numbers': ['123-456-7890'], 'education_info': ['John', '##e', 'Art', 'Shi', '##v', 'Na', '##dar', 'University']}
